In [1]:
import json
import visualize
import os
import context as c
import pandas as pd

In [2]:
def load(file_name):
    with open(f"Results/raw/{file_name}", "r") as f:
        res = json.load(f)
    return res

s_debiased = load("sdb_short.json")
l_debiased = load("sdb_long.json")
s_antonym = load("ant_short_probs.json")
l_antonym = load("ant_long_probs.json")

s_raw = visualize.get_sdb_df(s_debiased, c.context_t_i)
s_ant_raw = visualize.get_ant_prob_df(s_antonym, c.context_t_i)
l_raw = visualize.get_sdb_df(l_debiased, c.context_long_t_i)
l_ant_raw = visualize.get_ant_prob_df(l_antonym, c.context_long_t_i)

# Means

In [3]:
#s_raw.groupby(s_raw['Ethnicity']).mean().sort_values("Difference", ascending=False)
#visualize.get_sdb_means(s_raw, 'sdb_short_means.tex')

In [4]:
#l_raw.groupby(l_raw['Ethnicity']).mean().sort_values("Difference", ascending=False)
#visualize.get_sdb_means(l_raw, 'sdb_long_means.tex')

In [5]:
# means on word level
import pandas 
pandas.set_option('display.max_rows', 250)

In [6]:
# combine with antonyms

In [7]:
s_ant_comb = visualize.get_sdb_ant_df(s_raw, s_ant_raw)
l_ant_comb = visualize.get_sdb_ant_df(l_raw, l_ant_raw)

visualize.get_sdb_means(s_ant_comb, 'sdb_short_means.tex')
x = visualize.get_sdb_means(l_ant_comb, 'sdb_long_means.tex')

# get top n changes
visualize.get_top_n_changes(s_raw, file_name='sdb_s_top_10.tex')
visualize.get_top_n_changes(l_raw, file_name='sdb_l_top_10.tex')

# TODO should I do association scores as well?
# TODO go over antonyms
# TODO add control group? Or thats only for previous part of the work?
# TODO highlight out of vocab rows?

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:267: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:267: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(res.to_latex(index=False))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

,Ethnicity,Entity,Biased term,Translation,Original prob.,New prob,Difference,Change
169,Sami,man,tumma,dark,0.006446,0.005162,0.001283,19.91 %
159,Sami,woman,tumma,dark,0.013161,0.010587,0.002574,19.56 %
179,Sami,person,tumma,dark,0.006472,0.005261,0.001211,18.72 %
189,Sami,Elen,tumma,dark,0.004286,0.004135,0.000151,3.53 %
34,Finnish-Swedish,Laura,parempi,better,0.000712,0.000689,0.000023,3.24 %
44,Finnish-Swedish,Joel,parempi,better,0.000506,0.000491,0.000015,3.05 %
30,Finnish-Swedish,Laura,rikas,rich,0.000321,0.000314,0.000007,2.19 %
199,Sami,Hugo,tumma,dark,0.004480,0.004388,0.000093,2.07 %
36,Finnish-Swedish,Laura,terve,healthy,0.002460,0.002415,0.000045,1.82 %
40,Finnish-Swedish,Joel,rikas,rich,0.000492,0.000483,0.000009,1.82 %


In [16]:
print(len(s_ant_comb.loc[s_ant_comb['Difference'] <= 0])) # 87 rows with 0 difference
print(len(s_ant_comb.loc[s_ant_comb['Difference'] > 0])) # 163 rows with some difference
print(len(l_ant_comb.loc[l_ant_comb['Difference'] <= 0])) # 30 rows with 0 difference
print(len(l_ant_comb.loc[l_ant_comb['Difference'] > 0])) # 220 rows with some difference

87
163
30
220


In [8]:
visualize.get_sdb_ant_diff(s_ant_comb, "sdb_s_ant_diff.tex")
visualize.get_sdb_ant_diff(l_ant_comb, "sdb_l_ant_diff.tex")

/home/gussuvmi@GU.GU.SE/thesis/visualize.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(result.to_latex(index=index))
/home/gussuvmi@GU.GU.SE/thesis/visualize.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  file.write(result.to_latex(index=index))


,Original prob.,New prob,Antonym probability,Original difference,New difference
Ethnicity,,,,,
Finnish-Swedish,0.000233,0.000231,0.001435,-0.001202,-0.001205
Roma,0.000012,0.000012,0.000964,-0.000952,-0.000952
Russian,0.000024,0.000024,0.000479,-0.000455,-0.000455
Sami,0.000721,0.000614,0.000494,0.000226,0.000120
Somali,0.000026,0.000026,0.000505,-0.000479,-0.000479


In [9]:
new = s_raw.copy()
new['Change'] = new['Change']/100
new[['Original prob.', 'Change']].head(20).cov()

,Original prob.,Change
Original prob.,0.000694,0.002482
Change,0.002482,0.023791


In [10]:
l_cov = l_raw.copy()
l_cov['Change'] = l_cov['Change']/100
l_cov[['Original prob.', 'Change']].head(20).cov()

,Original prob.,Change
Original prob.,0.000011,0.000192
Change,0.000192,0.004404
